In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import os
#print(os.listdir("../input"))

In [2]:
import time
import gc

In [3]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFECV

/Users/ziluli/anaconda3/envs/elo/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [6]:
train = pd.read_csv('all/train.csv')

In [7]:
test = pd.read_csv('all/test.csv')

In [ ]:
#nmt = pd.read_csv('../input/new_merchant_transactions.csv')
#mct = pd.read_csv('../input/merchants.csv')

In [145]:
ht = pd.read_csv('all/historical_transactions.csv')

In [ ]:
data_dict = pd.read_excel('../input/Data_Dictionary.xlsx', sheet_name='train')
data_dict

In [ ]:
train.head()

In [146]:
def cal_time(x):
    if isinstance(x['first_active_month'], str):
        d1 = datetime.datetime.strptime(x['first_active_month'],'%Y-%m')
        d2 = datetime.datetime.strptime('2018-12','%Y-%m')
        delta = d2 - d1
    else:
        return float('nan')

    
    return int(delta.days/30)


In [147]:
train['time'] = train.apply(cal_time,axis=1)

In [148]:
test['time'] = test.apply(cal_time,axis=1)
test.loc[11578,'time'] = 22

In [ ]:
'''
train_X = train[['feature_1','feature_2','feature_3','time']]
train_y = train[['target']]
test['time'] = test.apply(cal_time,axis=1)
test_X = test[['feature_1','feature_2','feature_3','time']]
test.loc[11578,'time'] = 22
'''

# historical_transactions

In [ ]:
ht.info()

In [ ]:
ht.head()

In [ ]:
ht_dict = pd.read_excel('../input/Data_Dictionary.xlsx', sheet_name='history')
ht_dict

In [149]:
ht['authorized_flag'] = ht.authorized_flag.map({'Y':1,'N':0})

In [150]:
ht['category_1'] = ht.category_1.map({'Y':1,'N':0})

In [151]:
ht['category_2']=ht['category_2'].fillna(0)

In [152]:
ht['category_3'] = ht.category_3.map({'A':0,'B':1,'C':2,'nan':3})

In [153]:
ht['category_3']=ht['category_3'].fillna(3)

In [154]:
scaler = MinMaxScaler(copy = True, feature_range=(0,1))
scaler.fit(np.array(ht['purchase_amount']).reshape(-1,1))
ht['purchase_amount'] = scaler.transform(np.array(ht['purchase_amount']).reshape(-1,1))

In [155]:
import gc

In [156]:
ht.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,1,C_ID_4e6213e9bc,88,0,0,0.0,80,M_ID_e020e9b302,-8,7.250001e-09,2017-06-25 15:33:07,1.0,16,37
1,1,C_ID_4e6213e9bc,88,0,0,0.0,367,M_ID_86ec983688,-7,2.292501e-09,2017-07-15 12:10:45,1.0,16,16
2,1,C_ID_4e6213e9bc,88,0,0,0.0,80,M_ID_979ed661fc,-6,4.412501e-09,2017-08-09 22:04:29,1.0,16,37
3,1,C_ID_4e6213e9bc,88,0,0,0.0,560,M_ID_e6d5ae8ea6,-5,1.922500e-09,2017-09-02 10:06:26,1.0,16,34
4,1,C_ID_4e6213e9bc,88,0,0,0.0,80,M_ID_e020e9b302,-11,4.000000e-09,2017-03-10 01:14:19,1.0,16,37


In [157]:
def feature_eng11(data,prefix):
    data['purchase_date'] = pd.to_datetime(data['purchase_date'])
    data['purchase_month'] = data['purchase_date'].dt.month
    #data['month_diff'] = ((datetime.datetime.today() - data['purchase_date']).dt.days)//30
    #data['month_diff'] += data['month_lag']
    data['installments'] = data['installments'].astype(int)
    data.loc[:, 'purchase_date'] = pd.DatetimeIndex(data['purchase_date']). \
                                        astype(np.int64) * 1e-9
    data = pd.get_dummies(data, columns=['category_2', 'category_3'])
    agg_func = {
        'authorized_flag': ['sum', 'mean'],
        'category_1': ['sum', 'mean'],
        'category_2_0.0': ['mean', 'sum'],
        'category_2_1.0': ['mean', 'sum'],
        'category_2_2.0': ['mean', 'sum'],
        'category_2_3.0': ['mean', 'sum'],
        'category_2_4.0': ['mean', 'sum'],
        'category_2_5.0': ['mean', 'sum'],
        'category_3_0.0': ['sum', 'mean'],
        'category_3_1.0': ['sum', 'mean'],
        'category_3_2.0': ['sum', 'mean'],
        'category_3_3.0': ['sum', 'mean'],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'purchase_month': ['mean', 'max', 'min', 'std'],
        'purchase_date': [np.ptp, 'max', 'min'],
        'month_lag': ['min', 'max'],
        'merchant_category_id': ['nunique'],
        'state_id': ['nunique'],
        'subsector_id': ['nunique'],
        'city_id': ['nunique'],
    }
    agg_trans = data.groupby(['card_id']).agg(agg_func)
    agg_trans.columns = [prefix + '_'.join(col).strip() for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)

    df = (data.groupby('card_id')
          .size()
          .reset_index(name='{}transactions_count'.format(prefix)))

    agg_trans = pd.merge(df, agg_trans, on='card_id', how='left')

    return agg_trans

In [158]:
%%time
history = feature_eng11(ht, prefix='hist_')

CPU times: user 1min 47s, sys: 1min 5s, total: 2min 52s
Wall time: 2min 34s


In [159]:
history.head()

,card_id,hist_transactions_count,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_category_1_sum,hist_category_1_mean,hist_category_2_0.0_mean,hist_category_2_0.0_sum,hist_category_2_1.0_mean,hist_category_2_1.0_sum,...,hist_purchase_month_std,hist_purchase_date_ptp,hist_purchase_date_max,hist_purchase_date_min,hist_month_lag_min,hist_month_lag_max,hist_merchant_category_id_nunique,hist_state_id_nunique,hist_subsector_id_nunique,hist_city_id_nunique
0,C_ID_00007093c1,149,114,0.765101,28,0.187919,0.187919,28.0,0.000000,0.0,...,3.303308,32627654.0,1.519708e+09,1.487081e+09,-12,0,18,3,13,4
1,C_ID_0001238066,123,120,0.975610,2,0.016260,0.065041,8.0,0.772358,95.0,...,4.895850,13110825.0,1.519748e+09,1.506638e+09,-5,0,29,6,17,18
2,C_ID_0001506ef0,66,62,0.939394,0,0.000000,0.000000,0.0,0.030303,2.0,...,4.524190,34460275.0,1.518871e+09,1.484411e+09,-13,0,19,2,12,3
3,C_ID_0001793786,216,189,0.875000,2,0.009259,0.527778,114.0,0.050926,11.0,...,2.306373,24487497.0,1.509481e+09,1.484994e+09,-9,0,48,4,24,10
4,C_ID_000183fdda,144,137,0.951389,4,0.027778,0.027778,4.0,0.048611,7.0,...,4.439959,17492874.0,1.519592e+09,1.502099e+09,-6,0,36,7,21,9


In [160]:
temp = ht.groupby(['card_id']).apply(lambda x:x['city_id'].value_counts().idxmax())

In [161]:
temp2=ht.groupby(['card_id']).apply(lambda x:x['state_id'].value_counts().idxmax())

In [162]:
temp3=ht.groupby(['card_id']).apply(lambda x:x['subsector_id'].value_counts().idxmax())

In [163]:
abc = pd.DataFrame(temp,columns=['hist_city_id'])
history_ = pd.merge(history, abc, on='card_id', how='left')

In [164]:
abc = pd.DataFrame(temp2,columns=['hist_state_id'])
history_ = pd.merge(history_, abc, on='card_id', how='left')

In [165]:
abc = pd.DataFrame(temp3,columns=['hist_sub_id'])
history_ = pd.merge(history_, abc, on='card_id', how='left')

In [170]:
history_.head()

,card_id,hist_transactions_count,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_category_1_sum,hist_category_1_mean,hist_category_2_0.0_mean,hist_category_2_0.0_sum,hist_category_2_1.0_mean,hist_category_2_1.0_sum,...,hist_purchase_date_min,hist_month_lag_min,hist_month_lag_max,hist_merchant_category_id_nunique,hist_state_id_nunique,hist_subsector_id_nunique,hist_city_id_nunique,hist_city_id,hist_state_id,hist_sub_id
0,C_ID_00007093c1,149,114,0.765101,28,0.187919,0.187919,28.0,0.000000,0.0,...,1.487081e+09,-12,0,18,3,13,4,244,2,19
1,C_ID_0001238066,123,120,0.975610,2,0.016260,0.065041,8.0,0.772358,95.0,...,1.506638e+09,-5,0,29,6,17,18,314,9,19
2,C_ID_0001506ef0,66,62,0.939394,0,0.000000,0.000000,0.0,0.030303,2.0,...,1.484411e+09,-13,0,19,2,12,3,137,19,33
3,C_ID_0001793786,216,189,0.875000,2,0.009259,0.527778,114.0,0.050926,11.0,...,1.484994e+09,-9,0,48,4,24,10,179,-1,37
4,C_ID_000183fdda,144,137,0.951389,4,0.027778,0.027778,4.0,0.048611,7.0,...,1.502099e+09,-6,0,36,7,21,9,161,3,16


In [173]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,time,hist_transactions_count,hist_authorized_flag_sum,hist_authorized_flag_mean,...,hist_purchase_date_min,hist_month_lag_min,hist_month_lag_max,hist_merchant_category_id_nunique,hist_state_id_nunique,hist_subsector_id_nunique,hist_city_id_nunique,hist_city_id,hist_state_id,hist_sub_id
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,18,260,247,0.950000,...,1.498573e+09,-8,0,41,3,21,7,69,9,34
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,23,350,339,0.968571,...,1.483720e+09,-12,0,57,3,24,9,69,9,34
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,28,43,41,0.953488,...,1.484123e+09,-13,0,8,2,7,5,143,5,33
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,15,77,77,1.000000,...,1.506443e+09,-5,0,25,5,13,7,17,22,37
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,13,133,128,0.962406,...,1.510445e+09,-3,0,26,6,17,6,17,22,37


In [77]:
#history_ = history_.drop(columns=['hist_sub_id_x'])

In [172]:
gc.collect()
train = pd.merge(train, history_, on='card_id', how='left')
test = pd.merge(test, history_, on='card_id', how='left')
del history_
gc.collect()

42

In [101]:
len(train.columns)

108

In [174]:
del ht

# new_merchant_transactions

In [175]:
nmt = pd.read_csv('all/new_merchant_transactions.csv')

In [176]:
nmt['authorized_flag'] = nmt.authorized_flag.map({'Y':1,'N':0})
nmt['category_1'] = nmt.category_1.map({'Y':1,'N':0})
nmt['category_3'] = nmt.category_3.map({'A':0,'B':1,'C':2,'nan':3})
nmt['category_3']=nmt['category_3'].fillna(3)
nmt['purchase_date'] = pd.to_datetime(nmt['purchase_date'])

In [177]:
nmt.columns

Index(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments',
       'category_3', 'merchant_category_id', 'merchant_id', 'month_lag',
       'purchase_amount', 'purchase_date', 'category_2', 'state_id',
       'subsector_id'],
      dtype='object')

In [178]:
def aggregate_nmt(trans, prefix):
    # more features from this kernel: https://www.kaggle.com/chauhuynh/my-first-kernel-3-699
    trans['purchase_month'] = trans['purchase_date'].dt.month
    trans['year'] = trans['purchase_date'].dt.year
    trans['weekofyear'] = trans['purchase_date'].dt.weekofyear
    trans['month'] = trans['purchase_date'].dt.month
    trans['dayofweek'] = trans['purchase_date'].dt.dayofweek
    trans['weekend'] = (trans.purchase_date.dt.weekday >=5).astype(int)
    trans['hour'] = trans['purchase_date'].dt.hour
    trans['installments'] = trans['installments'].astype(int)
    #trans['month_diff'] = ((datetime.datetime.today() - trans['purchase_date']).dt.days)//30
    #trans['month_diff'] += trans['month_lag']

    trans.loc[:, 'purchase_date'] = pd.DatetimeIndex(trans['purchase_date']).astype(np.int64) * 1e-9
    trans['installments'] = trans['installments'].astype(int)
    trans = pd.get_dummies(trans, columns=['category_2', 'category_3'])
    agg_func = {
        'authorized_flag': ['sum', 'mean'],
        'category_1': ['sum', 'mean'],
        'category_2_1.0': ['mean', 'sum'],
        'category_2_2.0': ['mean', 'sum'],
        'category_2_3.0': ['mean', 'sum'],
        'category_2_4.0': ['mean', 'sum'],
        'category_2_5.0': ['mean', 'sum'],
        'category_3_0.0': ['sum', 'mean'],
        'category_3_1.0': ['sum', 'mean'],
        'category_3_2.0': ['sum', 'mean'],
        'category_3_3.0': ['sum', 'mean'],
        'merchant_id': ['nunique'],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'purchase_month': ['mean', 'max', 'min', 'std'],
        'purchase_date': [np.ptp, 'max', 'min'],
        'month_lag': ['min', 'max'],
        'merchant_category_id': ['nunique'],
        'state_id': ['nunique'],
        'subsector_id': ['nunique'],
        'city_id': ['nunique'],
    }
    agg_trans = trans.groupby(['card_id']).agg(agg_func)
    agg_trans.columns = [prefix + '_'.join(col).strip() for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)

    df = (trans.groupby('card_id')
          .size()
          .reset_index(name='{}transactions_count'.format(prefix)))

    agg_trans = pd.merge(df, agg_trans, on='card_id', how='left')

    return agg_trans

In [179]:
%%time
gc.collect()
new = aggregate_nmt(nmt, prefix='new_')
gc.collect()

CPU times: user 43.4 s, sys: 2.57 s, total: 45.9 s
Wall time: 40.9 s


In [180]:
new.head()

,card_id,new_transactions_count,new_authorized_flag_sum,new_authorized_flag_mean,new_category_1_sum,new_category_1_mean,new_category_2_1.0_mean,new_category_2_1.0_sum,new_category_2_2.0_mean,new_category_2_2.0_sum,...,new_purchase_month_std,new_purchase_date_ptp,new_purchase_date_max,new_purchase_date_min,new_month_lag_min,new_month_lag_max,new_merchant_category_id_nunique,new_state_id_nunique,new_subsector_id_nunique,new_city_id_nunique
0,C_ID_00007093c1,2,2,1,0,0.000000,0.500000,1,0.000000,0,...,0.000000,537024.0,1.523291e+09,1.522754e+09,2,2,2,2,2,2
1,C_ID_0001238066,26,26,1,2,0.076923,0.769231,20,0.000000,0,...,0.485165,5195343.0,1.525118e+09,1.519923e+09,1,2,15,4,9,8
2,C_ID_0001506ef0,2,2,1,0,0.000000,0.000000,0,0.000000,0,...,0.000000,471152.0,1.521710e+09,1.521239e+09,1,1,2,1,2,1
3,C_ID_0001793786,31,31,1,0,0.000000,0.483871,15,0.258065,8,...,0.475191,3981096.0,1.514742e+09,1.510761e+09,1,2,21,5,14,7
4,C_ID_000183fdda,11,11,1,0,0.000000,0.000000,0,0.000000,0,...,0.467099,5106807.0,1.525100e+09,1.519994e+09,1,2,9,2,6,2


In [181]:
temp = nmt.groupby(['card_id']).apply(lambda x:x['city_id'].value_counts().idxmax())
temp2 = nmt.groupby(['card_id']).apply(lambda x:x['state_id'].value_counts().idxmax())
temp3 = nmt.groupby(['card_id']).apply(lambda x:x['subsector_id'].value_counts().idxmax())
abc = pd.DataFrame(temp,columns=['new_city_id'])
new_ = pd.merge(new, abc, on='card_id', how='left')
abc = pd.DataFrame(temp2,columns=['new_state_id'])
new_ = pd.merge(new_, abc, on='card_id', how='left')
abc = pd.DataFrame(temp3,columns=['new_sub_id'])
new_ = pd.merge(new_, abc, on='card_id', how='left')

In [78]:
new_.head()

,card_id,newtransactions_count,newauthorized_flag_sum,newauthorized_flag_mean,newcategory_1_sum,newcategory_1_mean,newcategory_2_1.0_mean,newcategory_2_1.0_sum,newcategory_2_2.0_mean,newcategory_2_2.0_sum,...,newpurchase_date_min,newmonth_lag_min,newmonth_lag_max,newmerchant_category_id_nunique,newstate_id_nunique,newsubsector_id_nunique,newcity_id_nunique,new_city_id,new_state_id,new_sub_id
0,C_ID_00007093c1,2,2,1,0,0.000000,0.500000,1,0.000000,0,...,1.522754e+09,2,2,2,2,2,2,69,2,21
1,C_ID_0001238066,26,26,1,2,0.076923,0.769231,20,0.000000,0,...,1.519923e+09,1,2,15,4,9,8,314,9,37
2,C_ID_0001506ef0,2,2,1,0,0.000000,0.000000,0,0.000000,0,...,1.521239e+09,1,1,2,1,2,1,137,19,34
3,C_ID_0001793786,31,31,1,0,0.000000,0.483871,15,0.258065,8,...,1.510761e+09,1,2,21,5,14,7,69,9,37
4,C_ID_000183fdda,11,11,1,0,0.000000,0.000000,0,0.000000,0,...,1.519994e+09,1,2,9,2,6,2,161,3,16


In [183]:
gc.collect()
train = pd.merge(train, new_, on='card_id', how='left')
test = pd.merge(test, new_, on='card_id', how='left')
del new_
gc.collect()

14

In [184]:
len(train.columns)

108

***build model***

In [4]:
train2 = pd.read_csv('liwei_dataset/train_clean.csv')
test2 = pd .read_csv('liwei_dataset/test_clean.csv')

In [270]:
y = train['target']

In [5]:
train2.columns

Index(['first_active_month', 'card_id', 'feature_3', 'target', 'month', 'year',
       'elapsed_time', 'feature_1_1', 'feature_1_2', 'feature_1_3',
       ...
       'new_month_lag_min', 'new_month_lag_max', 'new_category_2_3.0_mean',
       'new_category_2_2.0_mean', 'new_purchase_date_ptp',
       'new_purchase_date_min', 'new_purchase_date_max',
       'new_category_3_B_mean', 'new_city_id_nunique', 'outliers'],
      dtype='object', length=121)

In [273]:
for col in ['first_active_month', 'card_id', 'target']:
    if col in train.columns:
        train.drop([col], axis=1, inplace=True)
    if col in test.columns:
        test.drop([col], axis=1, inplace=True)

In [274]:
X_train = train
X_test = test

In [275]:
len(X_test.columns)

117

In [276]:
n_fold = 5
kfolds = KFold(n_splits=n_fold, shuffle=True, random_state=11)

In [277]:
sfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=2333)

In [281]:
def train_model(X=X_train, X_test=X_test, y=y, params=None, folds=kfolds,model_type='lgb', plot_feature_importance=False):
    
    oof = np.zeros(len(X))
    prediction = np.zeros(len(X_test))
    scores = []
    feature_importance = pd.DataFrame()
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
        if model_type == 'lgb':
            model = lgb.LGBMRegressor(**params, n_estimators = 20000, nthread = 4, n_jobs = -1)
            ###  rfecv  ###
            selector = RFECV(estimator=model, cv=3)
            selector.fit(X_train, y_train)
            Good_features = []
            #get pruned set of features' list
            for i in range(len(selector.support_)):
                if selector.support_[i]==True:
                    Good_features.append(i)
            model.fit(X_train[X_train.columns[Good_features]], y_train, 
                    eval_set=[(X_train[X_train.columns[Good_features]], y_train), (X_valid[X_valid.columns[Good_features]], y_valid)], eval_metric='rmse',
                    verbose=1000, early_stopping_rounds=200)
            y_pred_valid = model.predict(X_valid[X_valid.columns[Good_features]])
            y_pred = model.predict(X_test[X_test.columns[Good_features]], num_iteration=model.best_iteration_)
            
        oof[valid_index] = y_pred_valid.reshape(-1,)
        scores.append(mean_squared_error(y_valid, y_pred_valid) ** 0.5)
        
        prediction += y_pred  
        if model_type == 'lgb':
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = X.columns
            fold_importance["importance"] = model.feature_importances_
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_fold
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    if model_type == 'lgb':
        feature_importance["importance"] /= n_fold
        if plot_feature_importance:
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12));
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
            plt.title('LGB Features (avg over folds)');
        
            return oof, prediction, feature_importance
        return oof, prediction
    
    else:
        return oof, prediction

In [282]:
params = {'num_leaves': 54,
         'min_data_in_leaf': 79,
         'objective': 'regression',
         'max_depth': 15,
         'learning_rate': 0.018545526395058548,
         "boosting": "gbdt",
         "feature_fraction": 0.8354507676881442,
         "bagging_freq": 3,
         "bagging_fraction": 0.8126672064208567,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         'min_child_weight': 5.343384366323818,
         'reg_alpha': 1.1302650970728192,
         'reg_lambda': 0.3603427518866501,
         'subsample': 0.8767547959893627,}

In [293]:
train.columns

Index(['feature_3', 'month', 'year', 'elapsed_time', 'feature_1_1',
       'feature_1_2', 'feature_1_3', 'feature_1_4', 'feature_1_5',
       'feature_2_1',
       ...
       'new_month_lag_min', 'new_month_lag_max', 'new_category_2_3.0_mean',
       'new_category_2_2.0_mean', 'new_purchase_date_ptp',
       'new_purchase_date_min', 'new_purchase_date_max',
       'new_category_3_B_mean', 'new_city_id_nunique', 'outliers'],
      dtype='object', length=118)

In [291]:
train.isna().sum()[train.isna().sum()>0]

unauthorized_transactions_count              30959
unauthorized_category_1_sum                  30959
unauthorized_category_1_mean                 30959
unauthorized_category_3_A_mean               30959
unauthorized_category_2_5.0_mean             30959
unauthorized_category_3_C_mean               30959
unauthorized_subsector_id_nunique            30959
unauthorized_merchant_id_nunique             30959
unauthorized_category_2_1.0_mean             30959
unauthorized_state_id_nunique                30959
unauthorized_category_2_4.0_mean             30959
unauthorized_purchase_amount_sum             30959
unauthorized_purchase_amount_mean            30959
unauthorized_purchase_amount_max             30959
unauthorized_purchase_amount_min             30959
unauthorized_purchase_amount_std             51988
unauthorized_installments_sum                30959
unauthorized_installments_mean               30959
unauthorized_installments_max                30959
unauthorized_installments_min  

In [283]:
oof_lgb, prediction_lgb, feature_importance = train_model(params=liwei_param, model_type='lgb', plot_feature_importance=True)

Fold 0 started at Fri Jan 18 10:21:52 2019


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [131]:
oof_lgb

array([-0.22376484, -0.38343586,  0.64515155, ..., -0.17424278,
       -2.5186664 , -0.01917684])

In [132]:
sample_submission = pd.read_csv('all/sample_submission.csv')
sample_submission['target'] = prediction_lgb
sample_submission.to_csv('lgb.csv', index=False)

In [113]:
train.to_csv('train_clean.csv',index=False)
test.to_csv('test_clean.csv',index=False)

In [125]:
temp = pd.read_csv('train_clean.csv')

In [128]:
len(temp.columns)

105

In [134]:
len(temp)

201917